In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.AIUtilities, 1.0.0-beta.23562.1"

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.AI, 1.0.0-beta.23567.4"


In [ ]:
#!value --name key
YOUR AZURE OPEN AI KEY

In [ ]:
#!value --name endpoint
https://your-enpoint.openai.azure.com/

# Use extension to connect to models

In [ ]:
#!connect azure-openai --model-type TextEmbeddingGenerator --kernel-name knowledge --api-key @value:key --endpoint @value:endpoint --deployment text-embedding-ada-002

In [ ]:
#!connect azure-openai --model-type ImageGenerator --kernel-name image --api-key @value:key --endpoint @value:endpoint --deployment DALL-E

In [ ]:
#!connect azure-openai --model-type ChatCompletion --kernel-name chat --api-key @value:key --endpoint @value:endpoint --deployment gpt-35-turbo-16k --use-knowledge knowledge

In [ ]:
System.Diagnostics.Debugger.Launch();

# 1. Hello...

### ... DALL-E

In [ ]:
A cat flying a red biplane, in the style of Hiyao Miyazaki.

#### *Points of interest* 

Extensions are able to provide custom visualizations. 

[Our code](https://github.com/jonsequitur/interactive/blob/semantic-kernel-kernel/src/Microsoft.DotNet.Interactive.OpenAI/SkiaUtils.cs)

[SkiaSharp's code](https://github.com/mono/SkiaSharp/blob/main/source/SkiaSharp.DotNet.Interactive/SkiaSharpKernelExtension.cs)

# 2. Skills

### Defining semantic functions for `Microsoft.SemanticKernel`

In [ ]:
#!function -h

In [ ]:
#!function summarize --skill writer
Summarize the following text. 
[BEGIN TEXT]
{{$input}}
[END TEXT]

In [ ]:
#!function stylist --skill writer
Rewrite the following text in the style of {{$person}}:
[BEGIN TEXT]
{{$input}}
[END TEXT]
Please write it in verse.

In [ ]:
#!set --name person --value @input:"Whose style would you like to summarize this in?"
#!use-skills function.writer.summarize function.writer.stylist
.NET Interactive engine can be used many ways.
* It takes messages, routes them to subkernels, and can format returned data.
* Messages are initiated via a host (like Polyglot Notebooks).
* Subkernels can be a bunch of different languages (including Mermaid).
* Subkernels can share data.
* Subkernels can be running in different processes and on different machines.
* We'd love your ideas on how we can leverage this for developers.


# 3. Kernels

### Using multiple AI models together

In [ ]:
#!function make_image_prompt --skill image
{{$input}}
Describe an image of the text above in {{$linecount}} lines.

In [ ]:
#!function safe --skill image
{{$input}}
Rewrite the text above to be ok to read with a 6 years old child.

In [ ]:
#!function improve --skill image
{{$input}}
Turn the text above into a better and more detailed prompt for DALL-E.

In [ ]:
#!set --name linecount --value 10
#!use-skills function.image.make_image_prompt function.image.safe function.image.improve 

A cat flying a red biplane, in the style of Hiyao Miyazaki.

In [ ]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.Events;

var prompts = new List<string>();

Kernel.Root.KernelEvents.Subscribe(e =>
{
    if (e is ReturnValueProduced rvp &&
        e.Command.TargetKernelName.Contains("chat(text)") &&
        rvp.FormattedValues.SingleOrDefault(v => v.MimeType == "text/plain") is { } plainTextValue)
    {
        prompts.Add(plainTextValue.Value);
    }
});

In [ ]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.AI;

var dallEKernel = Kernel.Root.FindKernels(k => k is ImageGenerationKernel).Single(k => k.Name == "image");

var lastPrompt = prompts.Last().Trim();
$"Sending this prompt to DALL-E: *{lastPrompt}*".DisplayAs("text/markdown");

var result = await dallEKernel.SendAsync(new SubmitCode(lastPrompt));

# 4. Bonus round

## `#!explain` redux

In [ ]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Events;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.AI;
using System.CommandLine;

var explainMagicCommand = new Command("#!explain");

Kernel.Root.AddDirective(explainMagicCommand);

In [ ]:
explainMagicCommand.SetHandler(async _ =>
{
    var context = KernelInvocationContext.Current;
    if (context.Command is SubmitCode submitCode)
    {
        var prompt = $"""
            {submitCode.Code.Replace("#!explain", "")}
            Explain what the code above does, using dad jokes.
            """;
        
        await Kernel.Root.FindKernelByName("chat(text)")
                    .SendAsync(new SubmitCode(prompt));

        context.Complete(submitCode);
    }
});


In [ ]:
#!explain
foreach (var i in Enumerable.Range(1, 10))
{
    if (i % 2 == 0)
    {
        Console.WriteLine(i);
    }
}

In [ ]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Events;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.AI;
using System.CommandLine;

var rewriteMagicCommand = new Command("#!rewrite");

Kernel.Root.AddDirective(rewriteMagicCommand);

In [ ]:
rewriteMagicCommand.SetHandler(async _ =>
{
    var context = KernelInvocationContext.Current;
    if (context.Command is SubmitCode submitCode)
    {
        var prompt = $"""
            {submitCode.Code.Replace("#!rewrite", "")}
            Rewrite the code above in python. just output the code, no markdown.
            """;
        
        await Kernel.Root.FindKernelByName("chat(text)")
                    .SendAsync(new SubmitCode(prompt));

        context.Complete(submitCode);
    }
});


In [ ]:
#!rewrite
foreach (var i in Enumerable.Range(1, 10))
{
    if (i % 2 == 0)
    {
        Console.WriteLine(i);
    }
}

In [ ]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Events;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.AI;
using System.CommandLine;

var unwindMagicCommand = new Command("#!unwind");

Kernel.Root.AddDirective(unwindMagicCommand);

In [ ]:
unwindMagicCommand.SetHandler(async _ =>
{
    var context = KernelInvocationContext.Current;
    if (context.Command is SubmitCode submitCode)
    {
        var prompt = $"""
            {submitCode.Code.Replace("#!unwind", "")}
            Rewrite the code above without recursion. Use iterations instead of recursion. just output the code, no markdown.
            """;
        
        await Kernel.Root.FindKernelByName("chat(text)")
                    .SendAsync(new SubmitCode(prompt));

        context.Complete(submitCode);
    }            
});

In [ ]:
#!unwind
int fibonacci(int x){
    switch (x){
        case 0: 
        return 1;
        case 1: 
        return 1;
        default:
        return fibonacci(x-1) + fibonacci(x-2);

    }
}

## Misc.

In [ ]:
#!set --name person --value @input:"Enter the name of the person you'd like to summarize this text."
#!use-skills function.writer.stylist

What is .NET Interactive?

It takes messages, routes them to subkernels, and can format returned data
Messages are initiated via a host
Subkernels can be a bunch of different languages (including Mermaid)
Subkernels can share data
Subkernels can be running in different processes or on different machines
We’d love your ideas on how we can leverage this for developers



In [ ]:
#!function explainer --skill writer
{{$input}} 

Summarize the text above in the style of Randall Munroe's Thing Explainer, using only words that are amongst the most common 500 english words


In [ ]:
#!function detect_intent 
Intent detection examples start here:
Intent: Unknown:
    * save
    * send
Intent: Help:
    * How can you help me?
    * What can you do?
Intent: TellAJoke:
    * What's the best joke you know?
    * Do you know any joke?
    * Tell me a joke
    * Tell me something funny
Intent: WhatTimeIsIt
    * tell me the time please
    * time and date please
    * what time is it?
    * what's the current time?
    * is it lunch time yet?
    * is it dinner time?
== end of examples ==
Input: {{$input}}
Intent:

In [ ]:
#!use-skills function._GLOBAL_FUNCTIONS_.detect_intent

I could really use some help understanding intent detection.

In [ ]:
#!use-skills function.writer.explainer

Since .NET Interactive is capable of running as a kernel for notebooks, it enables a polyglot (multi-language) notebook experience. When using the .NET Interactive kernel, you can use different languages from one cell to the next, share variables between languages, and dynamically connect new languages and remote kernels within a notebook. There's no need to install different Jupyter kernels, use wrapper libraries, or install different tools to get the best experience for the language of your choice. You can always use the best language for the job and seamlessly transition between different stages of your workflow, all within one notebook.

For the best experience when working with multi-language notebooks, we recommend installing the Polyglot Notebooks extension for Visual Studio Code. While the full .NET Interactive feature set is available in Jupyter, many features are only usable via code, whereas the Polyglot Notebooks extension provides additional features including a language/kernel picker for each cell, enhanced language services, a multi-kernel variable viewer, and more.